In [6]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher


In [7]:
def find_closest_name(name, other_names):
    best_score = 0
    best_name = None
    checked = []
    for other_name in other_names:
        if other_name in checked:
            continue
        if(isinstance(other_name, float)):
            continue
        score = SequenceMatcher(None, name, other_name).ratio()
        if score != None and score > best_score:
            best_score = score
            best_name = other_name
        checked.append(other_name)
    return best_name
def find_unmatches(years, league):
    unmatches = {}
    for y in years:
        table = pd.read_csv(league+"_data/"+league+y+'_with_form.csv')
        betting = pd.read_csv(league+"_data/"+league+y+'_bets.csv')
        teams = list(table['home_team_name'].values)
        bet_teams = list(betting['HomeTeam'].values)
        checked = []
        for t in teams:
            if(not isinstance(t, str)):
                continue
            if  t not in bet_teams and t not in checked:
                checked.append(t)
                unmatches[t] = find_closest_name(t, bet_teams)
        return unmatches


In [8]:
def add_betting(df, betting):
    for index, row in df.iterrows():
        # add the AvgA, AvgD, AvgH for home_team_name == HomeTeam and away_team_name === AwayTeam
        home_team_name = row['home_team_name']
        away_team_name = row['away_team_name']
        # Filter betting DataFrame to find corresponding rows
        if (not home_team_name in betting['HomeTeam'].values):
            print("Fix the names:",home_team_name)
        if (not away_team_name  in betting['AwayTeam'].values):
            print("Fix the names:",away_team_name)
        matching_row = betting[(betting['HomeTeam'] == home_team_name) & (betting['AwayTeam'] == away_team_name)]
        matching_row = matching_row.iloc[0]

        df.at[index, 'B365A'] = matching_row['B365A']
        df.at[index, 'B365D'] = matching_row['B365D']
        df.at[index, 'B365H'] = matching_row['B365H']



In [22]:
years = ['1415','1516','1718','1819','1920','2021','2122','2223']
leagues = ['bl','laliga','seriea']

for league in leagues:
    unmatch = find_unmatches(years,league)
    #isoteric instances
    unmatch['Darmstadt 98'] = 'Darmstadt'
    unmatch['Ingolstadt 04'] = 'Ingolstadt'
    unmatch['Düsseldorf'] = 'Fortuna Dusseldorf'
    unmatch['Nürnberg'] = 'Nurnberg'
    unmatch['Arminia'] = 'Bielefeld'
    unmatch['Greuther Fürth'] = 'Greuther Furth'
    unmatch['Sporting Gijón'] = 'Sp Gijon'
    unmatch['Leganés'] = 'Leganes'
    unmatch['Alavés'] = 'Alaves'
    unmatch['Cádiz'] = 'Cadiz'
    unmatch['SPAL'] = 'Spal'
    print(unmatch) # for sanity check
    
    for year in years:
        # first we open
        df = pd.read_csv(league+"_data/"+league+year+"_with_form.csv")
        bet = pd.read_csv(league+"_data/"+league+year+"_bets.csv")
        for um in unmatch.keys():
            bet['HomeTeam'] = bet['HomeTeam'].replace(unmatch[um], um)
            bet['AwayTeam'] = bet['AwayTeam'].replace(unmatch[um], um)
        add_betting(df, bet)
        df.to_csv(league+"_data/"+league+year+"_form_bets.csv")
        print('finished year', year)

{'Hannover 96': 'Hannover', 'Köln': 'FC Koln', 'Eint Frankfurt': 'Ein Frankfurt', 'Hertha BSC': 'Hertha', 'Paderborn 07': 'Paderborn', 'Gladbach': "M'gladbach", 'Hamburger SV': 'Hamburg', 'Mainz 05': 'Mainz', 'Darmstadt 98': 'Darmstadt', 'Ingolstadt 04': 'Ingolstadt', 'Düsseldorf': 'Fortuna Dusseldorf', 'Nürnberg': 'Nurnberg', 'Arminia': 'Bielefeld', 'Greuther Fürth': 'Greuther Furth', 'Sporting Gijón': 'Sp Gijon', 'Leganés': 'Leganes', 'Alavés': 'Alaves', 'Cádiz': 'Cadiz', 'SPAL': 'Spal'}
finished year 1415
finished year 1516
finished year 1718
finished year 1819
finished year 1920
finished year 2021
finished year 2122
finished year 2223
{'Málaga': 'Malaga', 'Almería': 'Almeria', 'Celta Vigo': 'Celta', 'Rayo Vallecano': 'Vallecano', 'Córdoba': 'Cordoba', 'Athletic Club': 'Ath Bilbao', 'Atlético Madrid': 'Ath Madrid', 'Espanyol': 'Espanol', 'La Coruña': 'La Coruna', 'Real Sociedad': 'Sociedad', 'Darmstadt 98': 'Darmstadt', 'Ingolstadt 04': 'Ingolstadt', 'Düsseldorf': 'Fortuna Dusseldor